# Chapter 2: Building an Interactive Job Dashboard

## Learning Objectives

By the end of this chapter, you will:
- Build a simple but effective interactive dashboard using real job data
- Create dynamic visualizations that respond to user input
- Implement basic data filtering features
- Add data export capabilities
- Apply dashboard design best practices

## Introduction to Dashboard Building

> **Instructor Cue:** Start by asking: "What makes a good dashboard? What would you want to see if you were analyzing job market data?" Emphasize that dashboards are powerful communication tools that make data accessible to non-technical users.

A dashboard transforms raw data into actionable insights through interactive visualizations. Today we'll build a focused job market dashboard that allows users to:

- Explore job opportunities by location and type
- Filter data based on different criteria
- Visualize key patterns
- Export data for further analysis

The goal is to create something simple yet powerful that demonstrates the core concepts of interactive data applications.

## Setting Up Our Dashboard

Let's start by creating a new Streamlit app specifically for our job dashboard. We will build a new file called `job_dashboard_app.py` in the `03_module/app` directory.

> **Instructor Cue:** Have everyone create this new file. Explain that we're using the cleaned, AI-enhanced dataset from Module 2 directly from where it was created.

In [ ]:
%%writefile app/job_dashboard_app.py

# <START> Streamlit setup
from datetime import datetime
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import streamlit as st

# Configure the page
st.set_page_config(
    page_title="Job Market Dashboard",
    page_icon="💼",
    layout="wide",
    initial_sidebar_state="expanded"
)

st.title("💼 Job Market Dashboard")
st.markdown("Explore job opportunities and market trends with interactive visualizations")

DATA_DIR = Path().cwd() / "02_module" / "data" / "indeed_jobs_cleaned.csv"
# <END> Streamlit setup

> **Instructor Cue:** Explain the `st.set_page_config()` function and its parameters. The `layout="wide"` parameter is particularly important for dashboards as it uses the full browser width.

## Loading and Preparing the Data

Now let's load our job data and add some basic data exploration features. Note that we're now using the enhanced dataset from Module 2, which includes AI-cleaned salary data, standardized job titles, and matched geographic locations:

In [ ]:
%%writefile -a app/job_dashboard_app.py

# <START> Data loading and caching
@st.cache_data
def load_job_data():
    """Load and prepare job data for dashboard"""

    try:
        # Load the cleaned and merged dataset from Module 2
        df = pd.read_csv(DATA_DIR)

        # The dataset already has cleaned salary columns (min_salary, max_salary)
        # We'll use the average of min/max for visualization purposes where both exist
        df['salary_numeric'] = df[['min_salary', 'max_salary']].mean(axis=1)

        # Use the cleaned location data from AI matching
        # Keep the original column names to match the actual dataset structure

        return df
    except FileNotFoundError as e:
        st.error("Job data file not found. Please ensure Module 2 data cleaning has been completed.")
        raise e

# Load the data
df = load_job_data()

if df is not None:
    st.success(f"✅ Loaded {len(df)} job listings successfully!")
else:
    st.stop()
# <END> Data loading and caching

> **Instructor Cue:** Explain the `@st.cache_data` decorator - this is crucial for performance. Without caching, the CSV would be reloaded every time someone interacts with the dashboard. Point out that we're now using the AI-enhanced dataset from Module 2, which gives us clean salary numbers, standardized job titles, and matched locations - this makes our dashboard much more powerful and accurate.

## Building Simple Dashboard Controls

Let's keep our filters simple and focused on the most important features:

In [ ]:
%%writefile -a app/job_dashboard_app.py

# <START> Simple sidebar controls
st.sidebar.header("🔍 Filter Jobs")

# State filter (simplified)
available_states = sorted(df['matched_state'].dropna().unique())
selected_state = st.sidebar.selectbox(
    "Select State:",
    options=['All'] + available_states,
    help="Choose a state to focus on"
)

# Occupation filter  
available_occupations = sorted(df['bls_title'].dropna().unique())
selected_occupation = st.sidebar.selectbox(
    "Select Job Type:",
    options=['All'] + available_occupations,
    help="Filter by job category"
)
# <END> Simple sidebar controls

> **Instructor Cue:** Point out how we're keeping the interface simple with just two key filters. This makes the dashboard easier to understand and use while still being interactive.

## Simple Data Filtering

Now let's apply our filters and show some key metrics:

In [ ]:
%%writefile -a app/job_dashboard_app.py

# <START> Apply filters
# Filter the data based on user selections
filtered_df = df.copy()

# Apply state filter
if selected_state != 'All':
    filtered_df = filtered_df[filtered_df['matched_state'] == selected_state]

# Apply occupation filter
if selected_occupation != 'All':
    filtered_df = filtered_df[filtered_df['bls_title'] == selected_occupation]

# Show summary metrics
col1, col2, col3 = st.columns(3)

with col1:
    st.metric("Total Jobs", len(filtered_df))

with col2:
    st.metric("Companies", filtered_df['company_name'].nunique())

with col3:
    avg_salary = filtered_df['salary_numeric'].mean()
    st.metric("Avg Salary", f"${avg_salary:,.0f}" if not pd.isna(avg_salary) else "N/A")

# <END> Apply filters

> **Instructor Cue:** Show how filtering updates the metrics in real-time. This demonstrates how Streamlit automatically reruns the code when inputs change.

## Two Key Visualizations

Let's create two simple but powerful charts that update based on our filters:

In [ ]:
%%writefile -a app/job_dashboard_app.py

# <START> Simple visualizations
if len(filtered_df) > 0:
    
    # Chart 1: Top Companies
    st.subheader("🏢 Top Hiring Companies")
    
    company_counts = filtered_df['company_name'].value_counts().head(8)
    if not company_counts.empty:
        fig, ax = plt.subplots(figsize=(10, 6))
        sns.barplot(x=company_counts.values, y=company_counts.index, hue=company_counts.index, ax=ax, palette='viridis', legend=False)
        ax.set_title('Companies with Most Job Postings')
        ax.set_xlabel('Number of Jobs')
        st.pyplot(fig)
        plt.clf()

    # Chart 2: Salary Distribution  
    st.subheader("💰 Salary Distribution")
    
    salary_data = filtered_df.dropna(subset=['salary_numeric'])
    if not salary_data.empty:
        fig, ax = plt.subplots(figsize=(10, 6))
        sns.histplot(data=salary_data, x='salary_numeric', bins=15, ax=ax, color='skyblue')
        ax.set_title('How Salaries Are Distributed')
        ax.set_xlabel('Salary ($)')
        ax.set_ylabel('Number of Jobs')
        ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))
        st.pyplot(fig)
        plt.clf()

else:
    st.warning("No jobs found. Try different filters!")

# <END> Simple visualizations

> **Instructor Cue:** Explain how these charts change dynamically as users adjust the filters. Point out the salary formatter that shows values like "$120K" instead of "$120000".

## Simple Data Table

Let's show the job data in a clean, simple table:

In [ ]:
%%writefile -a app/job_dashboard_app.py

# <START> Job listings table
st.subheader("📋 Job Listings")

# Show key columns in a simple table
display_columns = ['job_title', 'company_name', 'location', 'salary']
st.dataframe(filtered_df[display_columns].head(20), width='stretch')

# Simple download button
csv_data = filtered_df.to_csv(index=False)
st.download_button(
    label="📥 Download Data as CSV",
    data=csv_data,
    file_name=f"jobs_{datetime.now().strftime('%Y%m%d')}.csv",
    mime="text/csv"
)
# <END> Job listings table

> **Instructor Cue:** Point out how Streamlit makes it incredibly easy to display data with `st.dataframe()`. The download button shows how simple data export can be.

## Simple PDF Report Generation

Let's add a basic PDF report feature that includes a summary and one chart. First, install the required library:

Now let's create our simple report generator:

In [ ]:
%%writefile -a app/job_dashboard_app.py

# <START> Simple PDF Report
def create_simple_report(filtered_df):
    """Generate a simple PDF report with summary and one chart"""
    from fpdf import FPDF
    import tempfile
    import os
    
    class SimpleReport(FPDF):
        def header(self):
            self.set_font('helvetica', 'B', 16)
            self.cell(0, 10, 'Job Market Summary Report', new_x="LMARGIN", new_y="NEXT", align='C')
            self.ln(10)
        
        def footer(self):
            self.set_y(-15)
            self.set_font('helvetica', 'I', 8)
            self.cell(0, 10, f'Generated on {datetime.now().strftime("%Y-%m-%d")}', align='C')
    
    pdf = SimpleReport()
    pdf.add_page()
    
    # Summary text
    pdf.set_font('helvetica', '', 12)
    
    total_jobs = len(filtered_df)
    total_companies = filtered_df['company_name'].nunique()
    avg_salary = filtered_df['salary_numeric'].mean()
    avg_salary_text = f"${avg_salary:,.0f}" if not pd.isna(avg_salary) else "Not available"
    
    summary_text = f"""
        This report analyzes {total_jobs} job postings from {total_companies} different companies.

        Key Findings:
        1. Total job opportunities: {total_jobs}
        2. Number of hiring companies: {total_companies} 
        3. Average salary: {avg_salary_text}

        Top hiring companies are shown in the chart below.
    """
    
    pdf.multi_cell(0, 6, summary_text.strip())
    pdf.ln(10)
    
    # Create and save chart
    company_counts = filtered_df['company_name'].value_counts().head(5)
    if not company_counts.empty:
        fig, ax = plt.subplots(figsize=(8, 5))
        sns.barplot(x=company_counts.values, y=company_counts.index, hue=company_counts.index, ax=ax, palette='viridis', legend=False)
        ax.set_title('Top 5 Hiring Companies')
        ax.set_xlabel('Number of Jobs')
        
        # Save chart to temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as tmp:
            fig.savefig(tmp.name, bbox_inches='tight', dpi=150)
            chart_path = tmp.name
        
        plt.close(fig)
        
        # Add chart to PDF
        pdf.image(chart_path, x=10, w=180)
        
        # Clean up temp file
        os.unlink(chart_path)
    
    return bytes(pdf.output())

# PDF Download Section
st.subheader("📄 Simple Report")

if st.button("Generate PDF Report"):
    if len(filtered_df) > 0:
        with st.spinner("Creating report..."):
            pdf_data = create_simple_report(filtered_df)
            
            st.download_button(
                label="📥 Download PDF Report",
                data=pdf_data,
                file_name=f"job_report_{datetime.now().strftime('%Y%m%d')}.pdf",
                mime="application/pdf"
            )
            st.success("✅ Report ready for download!")
    else:
        st.warning("No data to report on. Try adjusting your filters.")
# <END> Simple PDF Report

> **Instructor Cue:** Explain how this creates a simple but professional PDF with text summary and one chart. Point out how we use temporary files to include the chart image in the PDF. Note that we use 'helvetica' font and modern fpdf2 syntax (new_x, new_y parameters) to avoid deprecation warnings.

## Adding a Simple Footer

Let's finish our dashboard with a simple footer:

In [ ]:
%%writefile -a app/job_dashboard_app.py

# <START> Simple footer
st.markdown("---")
st.markdown("*Job Market Dashboard - Built with Streamlit and Python*")
# <END> Simple footer

> **Instructor Cue:** Point out how we've created a complete interactive dashboard with just a few key components. The dashboard demonstrates the power of Streamlit for rapid data app development.

## Running Your Complete Dashboard

Save your notebook and run it with:

In [ ]:
import workshoplib.nbwidgets as nwb

nwb.StreamlitControlBtn("job_dashboard_app", 8501).render()
nwb.TunnelControlBtn(8501).render()

> **Instructor Cue:** Have everyone run their dashboard. Test the filters to see how the metrics and charts update dynamically.

## Testing Your Dashboard

Let's quickly verify the key features work:

1. **Try the filters** - Select different states and job types
2. **Watch the metrics** - See how totals change with filtering  
3. **View the charts** - Notice how visualizations update
4. **Download data** - Test the CSV export feature

## Key Takeaways

> **Instructor Cue:** Summarize the main learning points:

- **Interactive dashboards** make data accessible to non-technical users
- **Caching with `@st.cache_data`** improves performance significantly  
- **Simple filters** allow users to explore data effectively
- **Export capabilities** (CSV and PDF) make insights shareable beyond the dashboard

## Next Steps

> **Instructor Cue:** Preview the final module:

In Module 4, we'll integrate AI into our dashboard to create an intelligent job-matching agent that can:
- Extract skills from resumes
- Match candidates to the best job opportunities
- Provide AI-powered recommendations

The dashboard we built today will serve as the foundation for these AI features!